In [6]:
import numpy as np
import tensorflow as tf
import pickle as pkl
np.random.seed(0)
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Input, Dropout, LSTM, Activation, Bidirectional
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.initializers import glorot_uniform
np.random.seed(1)

In [7]:
datapath="C:/Users/aiden/Downloads/ProjectData/FeatureData/"
#with open(datapath+"Train", "rb") as f:
#    X_train, Y_train, _ = pkl.load(f)
with open(datapath+"Test", "rb") as f:
    X_test, Y_test, _ = pkl.load(f)
with open("C:/Users/aiden/Downloads/ProjectData/Vocab.pkl", "rb") as f:
    vocab = pkl.load(f)


In [8]:
vocab_size=len(vocab)

In [14]:
X_train_tensor=tf.ragged.constant(X_train, dtype=tf.float32)

In [17]:
print(tf.shape(X_train_tensor[:10,:,:]))

<DynamicRaggedShape lengths=[10, (43, 11, 39, 11, 13, 37, 59, 15, 15, 13), (7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7)] num_row_partitions=2>


In [20]:
def data_to_index (data):
    return [[vocab[w] for w in arr] for arr in data]

In [10]:
RELATION_VOCAB_SIZE=7

In [11]:
vocab_size

122

In [9]:
def CELoss(y_pred, x):
    on_stroke_indicator=x[:,:,6]
    #assumes that y_pred is log probabilities
    #log_prob_of_symbol = tf.math.reduce_logsumexp(y_pred[:,:,RELATION_VOCAB_SIZE:], axis=2)
    prob_of_symbol = tf.math.reduce_sum(y_pred[:,:,RELATION_VOCAB_SIZE:], axis=2)
    return(tf.math.reduce_sum(on_stroke_indicator*(-tf.math.log(prob_of_symbol))))

In [13]:
y_offstroke_vector=np.zeros(vocab_size)
y_offstroke_vector[RELATION_VOCAB_SIZE:]=0.1/(vocab_size-RELATION_VOCAB_SIZE)
y_offstroke_vector[:RELATION_VOCAB_SIZE]=0.9/RELATION_VOCAB_SIZE
y_onstroke_vector=np.zeros(vocab_size)
y_onstroke_vector[RELATION_VOCAB_SIZE:]=0.9/(vocab_size-RELATION_VOCAB_SIZE)
y_onstroke_vector[:RELATION_VOCAB_SIZE]=0.1/RELATION_VOCAB_SIZE

def get_y_pred(x, y_offstroke_vector, y_onstroke_vector):
    #x input shape = (batch_size, sequence_len, 7)
    y = np.zeros((x.shape[0], x.shape[1], y_offstroke_vector.shape[0]))
    y[x[:,:,6] == 1] =  y_onstroke_vector
    print(x[:,:,6]==1)
    y[x[:,:,6] == 0] = y_offstroke_vector
    return y
#CELoss Test
x=np.zeros((1, 5, 7))
x[:,:,6]=[1,0,1,0,1]
print(get_y_pred(x, y_offstroke_vector, y_offstroke_vector))

[[[0.12857143 0.12857143 0.12857143 0.12857143 0.12857143 0.12857143
   0.12857143 0.00086957 0.00086957 0.00086957 0.00086957 0.00086957
   0.00086957 0.00086957 0.00086957 0.00086957 0.00086957 0.00086957
   0.00086957 0.00086957 0.00086957 0.00086957 0.00086957 0.00086957
   0.00086957 0.00086957 0.00086957 0.00086957 0.00086957 0.00086957
   0.00086957 0.00086957 0.00086957 0.00086957 0.00086957 0.00086957
   0.00086957 0.00086957 0.00086957 0.00086957 0.00086957 0.00086957
   0.00086957 0.00086957 0.00086957 0.00086957 0.00086957 0.00086957
   0.00086957 0.00086957 0.00086957 0.00086957 0.00086957 0.00086957
   0.00086957 0.00086957 0.00086957 0.00086957 0.00086957 0.00086957
   0.00086957 0.00086957 0.00086957 0.00086957 0.00086957 0.00086957
   0.00086957 0.00086957 0.00086957 0.00086957 0.00086957 0.00086957
   0.00086957 0.00086957 0.00086957 0.00086957 0.00086957 0.00086957
   0.00086957 0.00086957 0.00086957 0.00086957 0.00086957 0.00086957
   0.00086957 0.00086957 0.0008695

In [33]:
def MathModel(input_shape, output_dim):
    input = Input(shape=input_shape)
    X = Bidirectional(LSTM(128, return_sequences = True))(input)
    X = Dropout(0.5)(X)
    X = Bidirectional(LSTM(128, return_sequences = True))(X)
    X = Dropout(0.5)(X)
    X = Bidirectional(LSTM(128, return_sequences = True))(X)
    X = Dropout(0.5)(X)
    X = Dense(output_dim+1)(X)
    output = Activation('softmax')(X)
    
    model = Model(inputs=input,outputs=output)
    model.add_loss(CELoss(output,input))
    
    return model

In [22]:
def CTCLoss(y_true, y_pred):
    # Compute the training-time loss value
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

    loss = tf.keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    return loss

In [28]:
indexed_y = data_to_index(Y_train)
indexed_y=tf.ragged.constant(indexed_y)

In [24]:
u

['B',
 'Sub',
 'n',
 'NoRel',
 '(',
 'Right',
 '1',
 'Right',
 '-',
 'Right',
 'x',
 'Right',
 ')',
 'Right',
 '=',
 'Right',
 '(',
 'Right',
 '-',
 'Right',
 '1',
 'Right',
 ')',
 'Sup',
 'n',
 'NoRel',
 'B',
 'Sub',
 'n',
 'NoRel',
 '(',
 'Right',
 'x',
 'Right',
 ')']

In [23]:
import math
class MathDataset(tf.keras.utils.Sequence):
    def __init__(self, x, y, batch_size, X_train_data_len, Y_train_data_len):
        self.x=x
        self.y=y
        self.batch_size=batch_size
        self.X_train_data_len=X_train_data_len
        self.Y_train_data_len=Y_train_data_len
    def __len__(self):
        return math.floor(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        max_x=max(self.X_train_data_len[idx * self.batch_size:(idx + 1) *
        self.batch_size])
        max_y=max(self.Y_train_data_len[idx * self.batch_size:(idx + 1) *
        self.batch_size])
        batch_x = self.x[idx * self.batch_size:(idx + 1) *
        self.batch_size, :max_x]
        batch_y = self.y[idx * self.batch_size:(idx + 1) *
        self.batch_size, :max_y]

        return (batch_x,batch_y)

NameError: name 'Y' is not defined

In [36]:
def trainModel(model, trainData, learning_rate, momentum, epochs):
    model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum), loss=CTCLoss)
    model.fit(trainData,epochs)
    return model


In [58]:
model = MathModel(input_shape=(None,7), output_dim=110)

In [57]:
model.compile(optimizer='adam', loss = CTCLoss, metrics = ['accuracy'])

In [ ]:
new_model=trainModel(model, X_train_tensor, indexed_y, 0.1, 0, 32, 2)

Epoch 1/2


TypeError: in user code:

    File "c:\Users\aiden\.conda\envs\tf\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\aiden\.conda\envs\tf\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\aiden\.conda\envs\tf\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\aiden\.conda\envs\tf\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\aiden\.conda\envs\tf\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None

    TypeError: Exception encountered when calling layer "tf.__operators__.getitem_2" "                 f"(type SlicingOpLambda).
    
    Failed to c

In [ ]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, None, 7)]    0           []                               
                                                                                                  
 bidirectional_11 (Bidirectiona  (None, None, 256)   139264      ['input_5[0][0]']                
 l)                                                                                               
                                                                                                  
 dropout_10 (Dropout)           (None, None, 256)    0           ['bidirectional_11[0][0]']       
                                                                                                  
 bidirectional_12 (Bidirectiona  (None, None, 256)   394240      ['dropout_10[0][0]']       